# Bibliotecas

In [1]:
import numpy as np
from tqdm import tqdm

# Abrindo dados

In [2]:
x = np.float64(np.load(open('data/cnn/x.npy', 'rb')))
y = np.int8(np.load(open('data/cnn/y.npy', 'rb')))

In [3]:
from keras.utils import to_categorical

y_one_hot = to_categorical(y, 3)
y_one_hot

2024-03-23 00:28:33.420812: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-23 00:28:33.467689: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-23 00:28:33.467718: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-23 00:28:33.468691: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-23 00:28:33.477073: I tensorflow/core/platform/cpu_feature_guar

array([[1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       ...,
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.]], dtype=float32)

# Criar Modelo

In [4]:
from keras.applications import EfficientNetV2L
from keras import layers, models, Sequential

input = layers.Input(shape=(501, 6, 1))
reshape_layer = layers.Cropping2D(cropping=((15, 0), (0, 0)))(input)
reshape_layer = layers.Reshape((54, 54, 1))(reshape_layer)
reshape_layer = layers.Concatenate()([reshape_layer, reshape_layer, reshape_layer])

importedNet = EfficientNetV2L(include_top=True, input_shape=(54, 54, 3), weights=None, classes=3)

modelo = importedNet(reshape_layer)
# modelo = layers.BatchNormalization()(modelo)
# modelo = layers.Flatten()(modelo)
# modelo = layers.Dropout(0.5)(modelo)
# modelo = layers.Dense(100, 'relu')(modelo)
# modelo = layers.Dropout(0.5)(modelo)
# modelo = layers.Dense(100, 'relu')(modelo)
# modelo = layers.Dense(3, 'softmax')(modelo)

modelo = models.Model(input, modelo)
modelo.compile(loss='categorical_crossentropy', metrics=['accuracy'])
modelo.summary()

2024-03-23 00:28:36.174833: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-23 00:28:36.198342: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-23 00:28:36.198399: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-23 00:28:36.201988: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-23 00:28:36.202040: I external/local_xla/xla/stream_executor

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 501, 6, 1)]          0         []                            
                                                                                                  
 cropping2d (Cropping2D)     (None, 486, 6, 1)            0         ['input_1[0][0]']             
                                                                                                  
 reshape (Reshape)           (None, 54, 54, 1)            0         ['cropping2d[0][0]']          
                                                                                                  
 concatenate (Concatenate)   (None, 54, 54, 3)            0         ['reshape[0][0]',             
                                                                     'reshape[0][0]',         

# Treinar Modelo

In [5]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard

tensorboard_callback = TensorBoard(log_dir="./logs/transferLearning/")
earlystop_cb = EarlyStopping(monitor='val_accuracy', patience=20, min_delta=0.001, verbose=True)
modelcheckpoint_cb = ModelCheckpoint('/mnt/c/Users/aurel/OneDrive/Documentos/Projetos/tradingPredictor/models/TransferLearningmodel5.h5', 'val_accuracy', save_best_only=True)

modelo.fit(
    x,
    y_one_hot,
    shuffle=True, 
    epochs=500,
    batch_size=256,
    validation_split=0.2,
    callbacks=[tensorboard_callback, earlystop_cb, modelcheckpoint_cb]
)

/home/aurelio/.local/lib/python3.10/site-packages/keras/src/initializers/__init__.py:144: UserWarning: The `keras.initializers.serialize()` API should only be used for objects of type `keras.initializers.Initializer`. Found an instance of type <class 'tensorflow.python.ops.init_ops_v2.Constant'>, which may lead to improper serialization.
  warnings.warn(
2024-03-23 00:28:52.654176: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


Epoch 1/500


2024-03-23 00:29:54.720077: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel/efficientnetv2-l/block1b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2024-03-23 00:30:03.718021: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902
2024-03-23 00:30:03.885122: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-03-23 00:30:18.744759: W external/local_tsl/tsl/framework/bfc_allocator.cc:485] Allocator (GPU_0_bfc) ran out of memory trying to allocate 5.2KiB (rounded to 5376)requested by op model/efficientnetv2-l/block5h_expand_bn/FusedBatchNormV3
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
202

ResourceExhaustedError: Graph execution error:

Detected at node model/efficientnetv2-l/block5h_expand_bn/FusedBatchNormV3 defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/home/aurelio/.local/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>

  File "/home/aurelio/.local/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/aurelio/.local/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/home/aurelio/.local/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/home/aurelio/.local/lib/python3.10/s